## ВМА. Лабораторная 5. Отчет
### Гамезо Валерия

### Постановка задачи

Разработать программу приведения матрицы к канонической форме Фробениуса методом Данилевского (регулярный случай); сохранить матрицы, используемые для получения собственных векторов. 

Для вычислений: 

- $n = 4$ – порядок матрицы;
- при заполнении матрицы $A$ использовать случайные числа из диапазона от $−50$ до $50$. Для получения случайных чисел использовать библиотечную функцию rand(), подключив хедер stdlib.h;
- использовать тип float; 
- «ведущий элемент» $a^{n - k}_{k, k - 1}$ на $(n–k+1)$-м шаге, $k=n, n–1,\dots, 2, (a^{(0)}_{n, n - 1} = a_{n, n - 1})$ должен быть отличным от нуля (рассматривается только такой регулярный случай); 
- считать $a^{(n - k)}_{k, k- 1}$ равным нулю, если $| a^{(n - k)}_{k, k- 1} | < 10^{-s}$;
- если $| a^{(n - k)}_{k, k- 1} | < 10^{-s}$, то выдать соответствующее сообщение и заново заполнить матрицу $A$. 

Программно реализовать для рассматриваемого примера алгоритм приведения матрицы к канонической форме Фробениуса. Порядок матрицы $n$ должен быть в коде параметром (это требование не является обязательным). Сохранить матрицы $M_{n – 1}, M_{n - 2}, \dots , M_1$, используемые для получения собственных векторов (понадобятся для другой работы). Вывести на печать матрицу $A$ (входные данные), полученную каноническую форму Фробениуса $Ф$, матрицы $M_{n – 1}, M_{n – 2}, \dots , M_1$, коэффициент $p_1$ (полученный из формы Фробениуса), след матрицы $\text{tr} A$ (для контроля вычислений: должно приближенно выполняться равенство $p_1 = \text{tr} A = a_{1, 1} + a_{2, 2}+ \dots + a_{n, n}$). 

### Входные данные

$$
A = \left( \begin{array}{rrrr}
  19 &  35 & -35 &  23 \\
 -50 &  48 & -40 & -30 \\
  32 & -41 & -38 & -17 \\
   6 &  32 & -16 &  23 \\
\end{array} \right)
$$

<center>
$\text{tr} A = 52$

### Листинг программы

Класс матрицы

```c++
class Matrix {
public:
    Matrix(const Matrix& matrix) : mSize(matrix.mSize) { //конструктор копирования
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];

            for (int j = 0; j < mSize; ++j) {
                a[i][j] = matrix[i][j];
            }
        }
    }
    
    Matrix(int size, bool isNull) : mSize(size) { // isNull - флаг, указывающий, заполнять матрицу нулями
                                                  // либо рандомными значениями (например, при инициализации 
                                                  // дополнительные матрицы M заполняются нулями)
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];
        }

        generateCoefs(isNull);
    }

    ~Matrix() {
        for (int i = 0; i < mSize; ++i) {
            delete[] a[i];
        }
    }


    float* operator[] (int index) const {
        return a[index];
    }
    
    int size() const {
        return mSize;
    }

    Matrix& operator* (const Matrix& other) { // оператор умножения матриц
        auto* result = new Matrix(mSize, true);

        for (int i = 0; i < mSize; ++i) {
            for (int j = 0; j < mSize; ++j) {
                for (int k = 0; k < mSize; ++k) {
                    (*result)[i][j] += a[i][k] * other[k][j];
                }
            }
        }

        return *result;
    } 
    
    Matrix& operator= (const Matrix& other) {
        if (this == &other) return *this;

        for (int i = 0; i < mSize; ++i) {
            for (int j = 0; j < mSize; ++j) {
                a[i][j] = other[i][j];
            }
        }

        return *this;
    }

private:
    void generateCoefs (bool isNull) { // генерация коеффициентов
        if (isNull) { // заполнение нулями
            for (int i = 0; i < mSize; ++i) {
                for (int j = 0; j < mSize; ++j) {
                    a[i][j] = 0;
                }
            }
        } else { // заполнение рандомными значениями в диапозоне [-50, 50]
            std::srand(std::time(0));

            for (int i = 0; i < mSize; ++i) {
                for (int j = 0; j < mSize; ++j) {
                    a[i][j] = std::rand() % 101 - 50;
                }
            }
        }
    }

    int mSize;
    float** a;
};
```

Класс, преобразующий начальную матрицу к нормальной форме Фробениуса

```c++
class FrobeniusForm{
public:

    explicit FrobeniusForm(const Matrix& exMatrix) : fSize(exMatrix.size()),
                                                     a(exMatrix) {
        Matrix cur(fSize, true);
        for (int i = 0; i < fSize - 1; ++i) { // инициализация дополнительных матриц
            m.push_back(cur);
            invertM.push_back(cur);
        }

        trace = getTrace(); // вычисление следа начальной матрицы

        state = "InputMatrix"; // состояние матрицы
    }
    float trace;

    bool getFNormalFrom() { // получение нормальной формы матрицы, возвращает true, если преобразование прошло удачно,
                            // иначе - false
        int i;
        for (i = fSize - 1; i > 0; --i) {
            if (fabs(a[i][i - 1]) > 1e-8) {
                generateMatriciesToMultiply(i - 1); // вычисление матриц M_i и M^{-1}_i
                a = invertM[i - 1] * a;
                a = a * m[i - 1];

            } else {
                return false;
            }
        }

        state = "FrobeniusForm";
        return true;
    }

private:
    int fSize; 
    std::string state; // строка состояния матрицы
    Matrix a; 
    std::vector<Matrix> m; // вектор матриц M
    std::vector<Matrix> invertM; // вектор матриц M^{-1}

    float getTrace() { // вычисление следа матриц
        float sum = 0;

        for (int i = 0; i < a.size(); ++i) {
            sum += a[i][i];
        }

        return sum;
    }
    
    void generateMatriciesToMultiply(int i) { // генерация дополнительных матриц
        for (int j = 0; j < fSize; ++j) {
            m[i][j][j] = 1;
            m[i][i][j] = -a[i + 1][j] / a[i + 1][i];

            invertM[i][j][j] = 1;
            invertM[i][i][j] = a[i + 1][j];

            if (i == j) {
                m[i][i][j] /= -a[i + 1][i];
            }
        }
    }
};
```

### Выходные данные

$$
M_3 = \left( \begin{array}{rrrr}
    1 &   0 &   0 &   0 \\
    0 &   1 &   0 &   0 \\
    0.375000 &   2.000000 &  -0.062500 &   1.437500 \\
    0 &   0 &   0 &   1 \\
\end{array} \right)
$$

$$
M_2 = \left( \begin{array}{rrrr}
    1 &   0 &   0 &   0 \\
    3.650078 &   0.001567 &  -0.122453 &   2.849334 \\
    0 &   0 &   1 &   0 \\
    0 &   0 &   0 &   1 \\
\end{array} \right)
$$

$$
M_1 = \left( \begin{array}{rrrr}
    0.000006 &  -0.001036 &   0.076253 &  -1.083492 \\
    0 &   1 &   0 &   0 \\
    0 &   0 &   1 &   0 \\
    0 &   0 &   0 &   1 \\
\end{array} \right)
$$

$$
F = \left( \begin{array}{rrrr}
       52 &     -813 &  -291468 &   841577 \\
        1 &        0 &        0 &        0 \\
        0 &        1 &        0 &        0 \\
        0 &        0 &        1 &        0 \\
\end{array} \right)
$$